Motivation: individual gross outliers from general station distribution are a common error in obs data by random recording, reporting, formatting, or instrumentation errors

Process:
1. uses individual observation deviations derived from monthly mean climatology calculated for each hour of the day
2. climatologies calculated using winsorised data to remove initial effect of outliers
    - Winsorising: all values beyond threhsold value from mean are set to that threshold value
    - 5 and 95% for hadisd
    - number of data values in population remains the same, not trimmed
3. raw unwinsorised observations are anomalised using these climatologies
4. standardized by IQR for that month and hour
    - IQR cannot be less than 1.5degC
5. values are low-pass filtered to remove any climate change signal causing overzealous removal at ends of time series
6. gaussian is fitted to the histogram of anomalies for each month
7. threshold value, rounded outwards where crosses y=0.1 line
8. distribution beyond threhsold value is scanned for gap, equal to bin width or more
9. all values beyond gap are flagged
10. obs that fall between critical threshold value and gap or critical threshold and end of distribution are tentatively flagged
    - these may be later reinstated on comparison with good data from neighboring stations

Notes:
- when applied to SLP, frequently flags storm signals, which may be of high interest, so this test is not applied to pressure data
- hadisd only applies to temp and dewpoint temp

In [1]:
import pandas as pd
import numpy as np
import xarray as xr

from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
ds = xr.open_dataset('/Users/victoriaford/Desktop/eaglerock/Historical Data Platform/Train_Files/ASOSAWOS_72051724165.nc')

df = ds.to_dataframe()
df = df.reset_index()
df['month'] = pd.to_datetime(df['time']).dt.month # sets month to new variable
df['year'] = pd.to_datetime(df['time']).dt.year # sets year to new variable

/Users/victoriaford/anaconda3/envs/histobs/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/Users/victoriaford/anaconda3/envs/histobs/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [3]:
def clim_mon_mean_hourly(df, var, month, hour):
    '''Calculate the monthly mean climatology for each of the day'''
    
    df_m_h = df.loc[(df.time.dt.month == month) & (df.time.dt.hour == hour)]
    clim_value = df_m_h[var].mean(numeric_only = True)
    
    # special handling if value is nan? 
    
    return clim_value

def iqr_range_monhour(df, var, month, hour):
    '''Calculates the monthly interquartile range per hour'''
    
    q1 = df.loc[(df.time.dt.month == month) & (df.time.dt.hour == hour)].quantile(0.25, numeric_only=True)
    q3 = df.loc[(df.time.dt.month == month) & (df.time.dt.hour == hour)].quantile(0.75, numeric_only=True)
    
    iqr_df = q3 - q1
    iqr_df_val = iqr_df[var]
    
    # iqr cannot be less than 1.5°C in order to preserve low variance stations
    if iqr_df_val < 1.5:
        iqr_df_val = 1.5
    else:
        iqr_df_val = iqr_df_val
            
    return iqr_df_val


def clim_standardized_anom(df, vars_to_anom):
    '''
    First anomalizes data by monthly climatology for each hour, then
    standardizes by the monthly climatological anomaly IQR for each hour
    '''
    
    df2 = df.copy()
    
    for var in vars_to_anom:
        for m in range(1,13,1):
            for h in range(0,24,1):
                # each hour in each month
                anom_value = clim_mon_mean_hourly(df, var, month=m, hour=h)
                iqr_value = iqr_range_monhour(df, var, month=m, hour=h)
                
                # locate obs within specific month/hour
                df_m_h = df.loc[(df.time.dt.month == m) & (df.time.dt.hour == h)]
                
                # calculate the monthly climatological anomaly by hour and standardize by iqr
                df2.loc[(df.time.dt.month == m) & 
                        (df.time.dt.hour == h), 
                        var] = (df_m_h[var] - anom_value) / iqr_value
                
    return df2

def winsorize_temps(df, vars_to_anom, winz_limits):
    '''
    Replaces potential spurious outliers by limiting the extreme values
    using the winz_limits set (default is 5% and 95% percentiles)
    '''
    
    df2 = df.copy()
    
    for var in vars_to_anom:
        for m in range(1,13,1):
            for h in range(0,24,1):
                
                df_m_h = df.loc[(df.time.dt.month == m) & (df.time.dt.hour == h)]
                
                # winsorize only vars in vars_to_anom
                df_w = winsorize(df_m_h[var], limits=winz_limits, nan_policy='omit')
                
                df2.loc[(df.time.dt.month == m) & (df.time.dt.hour == h),
                       var] = df_w
                
    return df2

In [ ]:
def low_pass_filter(df):
    '''
    Low pass filtering on observations to remove any climate change signal 
    causing overzealous removal at ends of time series
    '''
    
    

In [ ]:
def qaqc_climatological_outlier(df, winsorize=True, winz_limits=[0.05,0.05], plot=True, verbose=True):
    '''
    Flags individual gross outliers from climatological distribution.
    Only applied to air temperature and dew point temperature
    
    Input:
    ------
        df [pd.DataFrame]: station dataset converted to dataframe through QAQC pipeline
        plots [bool]: if True, produces plots of any flagged data and saved to AWS
            
    Returns:
    --------
        qaqc success:
            df [pd.DataFrame]: QAQC dataframe with flagged values (see below for flag meaning)
        qaqc failure:
            None
            
    Flag meaning:
    -------------
        25,qaqc_climatological_outlier,Value flagged as a climatological outlier
    '''
    
    #### ONLY IN NOTEBOOK DEVELOPMENT, REMOVED IN CODE FOR PIPELINE #####
    df = df.reset_index()
    df['month'] = pd.to_datetime(df['time']).dt.month # sets month to new variable
    df['year'] = pd.to_datetime(df['time']).dt.year # sets year to new variable
    
    vars_to_check = ['tas', 'tdps', 'tdps_derived']
    vars_to_anom = [v for v in vars_to_check if v in df.columns]
    
    # TO DO: filter to only use non-flagged data

    
    # winsorize data by percentiles
    if winsorize == True:
        df = winsorize_temps(df, vars_to_anom, winz_limits)
    else:
        df = df
        
    # standardize data by monthly climatological anomalies by hour
    df = clim_standardized_anom(df, vars_to_anom)

    # apply low pass filter
    
    # to remove any climate change signal causing overzealous removal at ends of time series
    
    # gaussian is fitted to the histogram of anomalies for each month
        # threshold value, rounded outwards where crosses y=0.1 line
        # distribution beyond threhsold value is scanned for gap, equal to bin width or more
        # all values beyond gap are flagged
            # HadISD: obs that fall between critical threshold value and gap or 
            # critical threshold and end of distribution are tentatively flagged
            # these may be later reinstated on comparison with good data from neighboring stations
            # into v2 of data product